<a href="https://colab.research.google.com/github/LucasBrandaoGomes/Projeto-ETL-Consultas-SQL/blob/main/G5_Consultas_sparkSQL_Completo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyspark

In [ ]:
pip install gcsfs

In [ ]:
import os
from google.cloud import storage

In [ ]:
serviceAccount = '/content/chave_projetofinal-grupo5-d2c4a9d78233.json'

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, dense_rank
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, FloatType

In [ ]:
from google.cloud import storage


def download_blob(bucket_name, source_blob_name, destination_file_name):

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)

    print("Downloaded storage object {} from bucket {} to local file {}."
         .format(source_blob_name, bucket_name, destination_file_name))


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
download_blob("projeto-final-bucket-g5", "saida/df_temp_co2_final.csv", "/content/drive/MyDrive/dados_soulcode/df_temp_co2_final_sql.csv")

Downloaded storage object saida/df_temp_co2_final.csv from bucket projeto-final-bucket-g5 to local file /content/drive/MyDrive/dados_soulcode/df_temp_co2_final_sql.csv.


In [ ]:
download_blob("projeto-final-bucket-g5", "saida/df_cflorestal_pyspark_final.csv", "/content/drive/MyDrive/dados_soulcode/df_cflorestal_sql.csv")

Downloaded storage object saida/df_cflorestal_pyspark_final.csv from bucket projeto-final-bucket-g5 to local file /content/drive/MyDrive/dados_soulcode/df_cflorestal_sql.csv.


In [ ]:
spark = (
    SparkSession.builder
      .master('local')
      .appName('dataframe_temp_co2')
      .config('spark.ui.port', '4050')
      .getOrCreate()
)

In [ ]:
df_temp_co2_final_sql = spark.read.format("csv") \
                             .option("inferSchema", True) \
                             .option("header", True) \
                             .option("sep", ",") \
                             .load("/content/drive/MyDrive/dados_soulcode/df_temp_co2_final_sql.csv")\
                             .createOrReplaceTempView("temp_co2") 

In [ ]:
df_cflorestal_sql = spark.read.format("csv") \
                             .option("inferSchema", True) \
                             .option("header", True) \
                             .option("sep", ",") \
                             .load("/content/drive/MyDrive/dados_soulcode/df_cflorestal_sql.csv")\
                             .createOrReplaceTempView("area_florestal") 

In [ ]:
#Query 1

# esta query serve para dar um panorama geral dos dados tratados e consolidados dos 
# dataframes trabalhados de emissões de CO2 e variações de temperatura dentro do período selecionado - 1900 a 2012

spark.sql('''
SELECT
  *
FROM
  temp_co2
''').show()

+-----------+------+----------+--------+--------------------+----+------------------+--------------------+----------------+-------------------+
|       Pais|Codigo|Continente|  Regiao|NivelDesenvolvimento| Ano|      TempMediaAno|     DifTempMediaAno|EmissaoCO2TonAno|DifEmissaoCO2TonAno|
+-----------+------+----------+--------+--------------------+----+------------------+--------------------+----------------+-------------------+
|Afghanistan|   AFG|      Ásia|Ásia Sul|  Em desenvolvimento|1949|13.350083380937576| -0.9851666490236912|         14656.0|               null|
|Afghanistan|   AFG|      Ásia|Ásia Sul|  Em desenvolvimento|1950|13.043499952182174|-0.30658342875540257|         84272.0|            69616.0|
|Afghanistan|   AFG|      Ásia|Ásia Sul|  Em desenvolvimento|1951|13.967749883731207|  0.9242499315490331|         91600.0|             7328.0|
|Afghanistan|   AFG|      Ásia|Ásia Sul|  Em desenvolvimento|1952|14.175416549046835| 0.20766666531562805|         91600.0|             

In [ ]:
#Query 2

# esta query serve para visualizarmos quais os maiores emissores de CO2 dentro do período do nosso recorte - de 1900 a 2012
# mostrando as emissões máximas de cada um neste histórico


spark.sql('''
SELECT 
  Pais, 
  Continente, 
  Regiao, 
  MAX(EmissaoCO2TonAno) AS maxEmissaoCO2TonAno
FROM 
  temp_co2
GROUP BY 
  Pais, 
  Continente, 
  Regiao
ORDER BY 
  maxEmissaoCO2TonAno DESC
''').show()

+--------------+----------+--------------------+-------------------+
|          Pais|Continente|              Regiao|maxEmissaoCO2TonAno|
+--------------+----------+--------------------+-------------------+
|         China|      Ásia|          Ásia Leste|        9.6338995E9|
| United States|   América|    América do Norte|        6.1320059E9|
|        Russia|    Europa|        Europa Leste|       2.57121075E9|
|         India|      Ásia|            Ásia Sul|       1.98375885E9|
|         Japan|      Ásia|          Ásia Leste|       1.30208294E9|
|       Germany|    Europa|        Europa Oeste|       1.12005184E9|
|       Ukraine|    Europa|        Europa Leste|        7.6753971E8|
|United Kingdom|    Europa|        Europa Norte|        6.6038829E8|
|     Indonesia|      Ásia|        Ásia sudeste|        6.3314502E8|
|          Iran|      Ásia|            Ásia Sul|        6.0068032E8|
|        Canada|   América|    América do Norte|        5.9414022E8|
|   South Korea|      Ásia|       

In [ ]:
#Query 3

# esta query mostra os valores máximos de temperatura média anual atingida em cada país dentro do recorte histórico de 1900 a 2012
# permitindo que vejamos os países mais quentes/frios e poder vislumbrar quais seriam os impactos de uma variação de temperatura nestes locais

spark.sql('''
SELECT 
  Pais, 
  Continente, 
  Regiao, 
  MAX(TempMediaAno) AS maxTempMediaAno
FROM 
  temp_co2
GROUP BY 
  Pais, 
  Continente, 
  Regiao
ORDER BY 
  maxTempMediaAno DESC
''').show()

+--------------------+----------+--------------+------------------+
|                Pais|Continente|        Regiao|   maxTempMediaAno|
+--------------------+----------+--------------+------------------+
|                Mali|    África|  África Oeste| 30.12708346048991|
|            Djibouti|    África|  África Leste|29.923583189646404|
|United Arab Emirates|      Ásia|    Ásia Oeste| 29.67133315404256|
|          Mauritania|    África|  África Oeste| 29.37291733423869|
|             Senegal|    África|  África Oeste|29.302333037058514|
|               Niger|    África|  África Oeste|29.299916744232178|
|        Burkina Faso|    África|  África Oeste|29.288750012715656|
|               Aruba|   América|América Caribe|28.982499917348225|
|               Qatar|      Ásia|    Ásia Oeste|28.889583269755047|
|              Gambia|    África|  África Oeste|28.826083183288574|
|                Chad|    África| África Centro|28.696499824523926|
|               Sudan|    África|  África Norte|

In [ ]:
#Query 4

# esta query permite uma análise, dentre os 5 continentes*, da contribuição de cada um para as emissões de CO2 a nível global
# ordenados do maior responsável ao menor 
# *esta divisão foi feita pelo grupo do Projeto 5 com base em dados da ONU

spark.sql('''
SELECT 
  Continente, 
  SUM(EmissaoCO2TonAno) AS emissaoContinente
FROM 
  temp_co2
GROUP BY 
  Continente
ORDER BY 
  emissaoContinente DESC
''').show()

+----------+--------------------+
|Continente|   emissaoContinente|
+----------+--------------------+
|   América|  4.4717635006159E11|
|    Europa|4.339716828486199E11|
|      Ásia|3.714579749650603E11|
|    África|3.640263522689999...|
|   Oceania|1.714756707880000...|
+----------+--------------------+



In [ ]:
#Query 5

# esta query permite uma análise, dentre as 22 regiões do planeta*, da contribuição de cada uma para as emissões de CO2 a nível global
# ordenados da principal região na contribuição das emissões de CO2 - América do Norte - à que menos contribuiu - Micronésia
# é interessante notar, nesta tabela, os maiores e menores contribuintes em termos de emissões de CO2 -  
# 1º - América do Norte, 2º - Ásia Leste, 3º - Europa Leste, 4º - Europa Oeste - todos na casa das centenas de bilhões de ton de Emissões de CO2 por Ano
# 19º - África Centro, 20º - Melanesia, 21º - Polynesia, 22º - Micronésia - 
# os dois primeiros apontados na casa das centenas de milhões e os dois últimos na casa das dezenas de milhões de ton de Emissões de CO2 por Ano

# *esta divisão foi feita pelo grupo do Projeto 5 com base em dados da ONU

spark.sql('''
SELECT 
  Regiao, 
  SUM(EmissaoCO2TonAno) AS emissaoRegiao
FROM 
  temp_co2
GROUP BY 
  Regiao
ORDER BY 
  emissaoRegiao DESC
''').show(25)

+--------------------+--------------------+
|              Regiao|       emissaoRegiao|
+--------------------+--------------------+
|    América do Norte|    3.91133958922E11|
|          Ásia Leste|2.332077219538999...|
|        Europa Leste|   1.709119566871E11|
|        Europa Oeste|  1.4107174649375E11|
|        Europa Norte|    7.66388442554E10|
|            Ásia Sul|5.550703901760998...|
|          Europa Sul|4.534913541236999E10|
|          Ásia Oeste|    3.79150376912E10|
|      América do Sul|   3.373184480055E10|
|        Ásia sudeste|2.612035077564999...|
|        Ásia Central|1.870782552670000...|
|     América Central|   1.775342567061E10|
|          África Sul|    1.76355191964E10|
|Australia e Nova ...|     1.6828292053E10|
|        África Norte|1.185012446300000...|
|      América Caribe| 4.557120668430002E9|
|        África Oeste| 3.950222254939999E9|
|        África Leste|1.9996711723500004E9|
|       África Centro| 9.670981402099999E8|
|           Melanesia| 2.7370219

In [ ]:
#Query 6

# esta tabela nos permite observar que, embora representem apenas um quarto (aproximadamente) do total de países listados do grupo analisado
# os países Desenvolvidos ainda assim possuem mais do dobro de emissões de CO2 que os países Em desenvolvimento
# e ainda contando com o fato de que a tendência histórica de variação de Emissão de CO2 
# é quase o dobro nos países Em desenvolvimento quando comparada nos páises Desenvolvidos,
# tudo isso dentro do período compreendido entre 1900 e 2012

spark.sql('''
SELECT 
  NivelDesenvolvimento, 
  COUNT(DISTINCT Pais) AS qtdPaises, 
  SUM(EmissaoCO2TonAno) AS emissaoNivelDesenvolvimento, 
  ROUND(SUM(DifEmissaoCO2TonAno),5) AS varEmissaoCO2 
FROM 
  temp_co2
GROUP BY 
  NivelDesenvolvimento
ORDER BY 
  EmissaoNivelDesenvolvimento DESC
''').show()

+--------------------+---------+---------------------------+--------------------+
|NivelDesenvolvimento|qtdPaises|emissaoNivelDesenvolvimento|       varEmissaoCO2|
+--------------------+---------+---------------------------+--------------------+
|        Desenvolvido|       51|         8.9999636159636E11|  1.0644601432925E10|
|  Em desenvolvimento|      153|       4.061598485846101E11|1.973777451925355E10|
+--------------------+---------+---------------------------+--------------------+



In [ ]:
# Query 7

# esta query nos permite observar os países e regiões em que houve maior variação de temperatura acumulada 
# dentro do período observado, apresentando dados com valores bastante significativos
# as cinco primeiras linhas retornam valores de  variação de temperatura acima de 2 ºC positivos em um período de 113 anos

spark.sql('''
SELECT 
  Pais,
  Continente,
  Regiao, 
  SUM(DifTempMediaAno) AS totalVarTempMedia
FROM 
  temp_co2
GROUP BY 
  Pais,
  Continente,
  Regiao
ORDER BY 
  totalVarTempMedia DESC
''').show(200) 

+--------------------+----------+--------------------+--------------------+
|                Pais|Continente|              Regiao|   totalVarTempMedia|
+--------------------+----------+--------------------+--------------------+
|           Greenland|   América|    América do Norte|  2.3875830620527267|
|             Ireland|    Europa|        Europa Norte|  2.3140000502268467|
|              Russia|    Europa|        Europa Leste|  2.2523331046104436|
|              Jordan|      Ásia|          Ásia Oeste|  2.2376667658487968|
|             Estonia|    Europa|        Europa Norte|   2.208416510373354|
|              Latvia|    Europa|        Europa Norte|  2.1617499006291228|
|             Bahrain|      Ásia|          Ásia Oeste|     2.0809166431427|
|              Norway|    Europa|        Europa Norte|   2.071750046064455|
|             Uruguay|   América|      América do Sul|   1.912750005722046|
|             Finland|    Europa|        Europa Norte|   1.906750084211429|
|           

In [ ]:
#Query 8

# esta query permite que busquemos uma possível relação entre Emissão de CO2 e variação de temperatura média dos diferentes Países
# dando atenção também à localização geográfica - Continente e Região
# no período observado - 1900 a 2012

spark.sql('''
SELECT 
  Pais, 
  Continente, 
  Regiao, 
  SUM(EmissaoCO2TonAno) AS somaEmissaoCO2, 
  ROUND(SUM(DifTempMediaAno),5) AS somaDifTempMedia
FROM 
  temp_co2
GROUP BY 
  Pais, 
  Continente, 
  Regiao
ORDER BY 
  somaEmissaoCO2 DESC
''').show(15)

+--------------+----------+--------------------+---------------+----------------+
|          Pais|Continente|              Regiao| somaEmissaoCO2|somaDifTempMedia|
+--------------+----------+--------------------+---------------+----------------+
| United States|   América|    América do Norte|3.6228707408E11|          1.2395|
|         China|      Ásia|          Ásia Leste|1.5125980995E11|         0.53958|
|        Russia|    Europa|        Europa Leste| 9.222324485E10|         2.25233|
|       Germany|    Europa|        Europa Oeste|8.0477050946E10|         0.77508|
|United Kingdom|    Europa|        Europa Norte|5.8634289928E10|         0.46142|
|         Japan|      Ásia|          Ásia Leste|5.5932726122E10|           1.092|
|         India|      Ásia|            Ásia Sul|3.7119184284E10|           0.273|
|        France|    Europa|        Europa Oeste|3.2414488784E10|         0.65917|
|        Canada|   América|    América do Norte|2.8819349882E10|         1.84192|
|       Ukraine|

In [ ]:
#Query 9

# esta query permite que busquemos uma relação entre Emissão de CO2 e variação de temperatura média dos diferentes Países
# a partir da ótica das variações de temperaturas, trazendo novas possibilidades de insights
# também dando atenção à localização geográfica - Continente e Região
# no período observado - 1900 a 2012

spark.sql('''
SELECT 
  Pais, 
  Continente, 
  Regiao, 
  SUM(EmissaoCO2TonAno) AS somaEmissaoCO2, 
  ROUND(SUM(DifTempMediaAno),5) AS somaDifTempMedia
FROM 
  temp_co2
GROUP BY 
  Pais, 
  Continente, 
  Regiao
ORDER BY 
  somaDifTempMedia DESC
''').show(15)

+---------+----------+----------------+--------------------+----------------+
|     Pais|Continente|          Regiao|      somaEmissaoCO2|somaDifTempMedia|
+---------+----------+----------------+--------------------+----------------+
|Greenland|   América|América do Norte|          2.753496E7|         2.38758|
|  Ireland|    Europa|    Europa Norte|        1.91959327E9|           2.314|
|   Russia|    Europa|    Europa Leste|      9.222324485E10|         2.25233|
|   Jordan|      Ásia|      Ásia Oeste|4.9917782146000004E8|         2.23767|
|  Estonia|    Europa|    Europa Norte|       1.055969029E9|         2.20842|
|   Latvia|    Europa|    Europa Norte|       5.820661835E8|         2.16175|
|  Bahrain|      Ásia|      Ásia Oeste|        6.62789493E8|         2.08092|
|   Norway|    Europa|    Europa Norte|       2.300883789E9|         2.07175|
|  Uruguay|   América|  América do Sul|       3.186701215E8|         1.91275|
|  Finland|    Europa|    Europa Norte|       2.821285606E9|    

In [ ]:
#Query 10

# esta query apresenta uma análise do acumulado histórico das emissões de CO2 e variação total de temperatura
# dentro do período considerado - 1900 a 2012, sendo que esta faixa de tempo é menor para alguns páises
# devido a questões socio-políticas
# justamente numa das regiões mais afetadas pelas mudanças climáticas - a Ásia Oeste/Oriente Médio

spark.sql('''
SELECT 
  Pais, 
  Regiao, 
  ROUND(SUM(DifTempMediaAno),5) AS somaDifTempMedia, 
  ROUND(SUM(EmissaoCO2TonAno),5) AS somaEmissaoCO2
FROM 
  temp_co2
GROUP BY 
  Pais, 
  Regiao
HAVING 
  Regiao = 'Ásia Oeste'
ORDER BY 
  somaDifTempMedia DESC
''').show(20)

+--------------------+----------+----------------+----------------+
|                Pais|    Regiao|somaDifTempMedia|  somaEmissaoCO2|
+--------------------+----------+----------------+----------------+
|              Jordan|Ásia Oeste|         2.23767|  4.9917782146E8|
|             Bahrain|Ásia Oeste|         2.08092|    6.62789493E8|
|              Kuwait|Ásia Oeste|         1.90425|   2.124607404E9|
|               Qatar|Ásia Oeste|         1.87042|   1.318490569E9|
|              Cyprus|Ásia Oeste|         1.77942|  2.3582737574E8|
|                Iraq|Ásia Oeste|         1.61675|  3.1070403535E9|
|          Azerbaijan|Ásia Oeste|         1.43983|   2.280646502E9|
|United Arab Emirates|Ásia Oeste|         1.40683|   3.085029108E9|
|             Armenia|Ásia Oeste|          1.3805|   2.451920716E8|
|              Israel|Ásia Oeste|         1.32408|   1.893874275E9|
|               Yemen|Ásia Oeste|         1.31675|   5.137040065E8|
|                Oman|Ásia Oeste|         1.2912

In [ ]:
#Query 11 (BR)

# esta query mostra as Emissões de CO2 e variações de temperatura ano a ano para o caso específico do Brasil
# é possível observar que não há correlação direta, no caso brasileiro, entre esses dois fatores analisados
# pois o país apresenta uma taxa cada vez mais elevada de emissões e uma variação muito pequena de temperatura
# é notável também que o crescimento no valor de emissões se dá quase que linearmente com a passagem dos anos,
# aumentando progressivamente 

spark.sql('''
SELECT 
  Pais, 
  Ano, 
  ROUND(DifTempMediaAno, 8) AS varTempMediaAno, 
  EmissaoCO2TonAno
FROM 
  temp_co2
WHERE 
  Pais = 'Brazil'
ORDER BY 
  EmissaoCO2TonAno DESC
''').show(25)

+------+----+---------------+----------------+
|  Pais| Ano|varTempMediaAno|EmissaoCO2TonAno|
+------+----+---------------+----------------+
|Brazil|2012|       0.288167|    4.60063104E8|
|Brazil|2011|    -0.38349994|    4.30194912E8|
|Brazil|2010|     0.21183316|    4.11240192E8|
|Brazil|2008|    -0.25766643|    3.80344928E8|
|Brazil|2009|     0.19025008|    3.60090784E8|
|Brazil|2007|      0.1509997|    3.56918752E8|
|Brazil|2005|     0.30950022|    3.42086464E8|
|Brazil|2006|    -0.28900019|    3.41946368E8|
|Brazil|2004|     -0.1244998|    3.33675136E8|
|Brazil|2001|      0.3079168|    3.32987392E8|
|Brazil|2002|     0.29591703|    3.27434816E8|
|Brazil|2000|    -0.09008344|    3.24226048E8|
|Brazil|2003|    -0.17933385|    3.17508448E8|
|Brazil|1999|    -0.67216682|    3.16282432E8|
|Brazil|1998|     0.39933364|    3.08305728E8|
|Brazil|1997|     0.22158321|    2.96585248E8|
|Brazil|1996|    -0.21658325|    2.81194784E8|
|Brazil|1995|     0.09974972|    2.55583376E8|
|Brazil|1994|

In [ ]:
#Criação de Tabela brasil_1990_2010 no BigQuery

# esta tabela fornece um recorte específico de país - Brasil - e período de tempo - 1990 a 2010
# para que possamos fazer análises cruzadas com os dados sobre área de cobertura florestal

'''
CREATE OR REPLACE TABLE
  brasil_1990_2010 AS (
  SELECT
    Pais,
    Ano,
    ROUND(DifTempMediaAno, 8) AS varTempMediaAno,
    EmissaoCO2TonAno
  FROM
    temp_co2
  WHERE
    Pais = 'Brazil'
    AND Ano > 1989
    AND Ano < 2011
  ORDER BY
    EmissaoCO2TonAno DESC )
'''

In [ ]:
#Query 12

# esta query nos permite observar, por meio da combinação de dados da tabela com recorte específico para o Brasil entre 1990 e 2010
# e a tabela de cobertura florestal no mundo entre 1990 e 2010, um aumento significativo na temperatura média do período considerado
# somado a uma crescente no total de Emissões de CO2  - atingindo o volume aproximado de 6 bilhões de toneladas em emissões - em um
# período em que perdemos o equivalente à área aproximada do Estado do Rio de Janeiro em cobertura florestal
# o que aponta para uma possível correlação entre os três tipos de dados considerados nessa análise:
# Variação de Temperatura Média, Total de Emissões de CO2 e Área de Cobertura Florestal

'''
SELECT
  a.Pais,
  SUM(varTempMediaAno) AS VarTotalTempMedia,
  SUM(EmissaoCO2TonAno) AS TotalEmissoesCO2,
  b.AreaFlorestal_1990_1000ha_,
  b.AreaFlorestal_2010_1000ha_,
  b.Variacao_1990_2010,
  b.VarPercentual_1990_2010
FROM
  `projetofinal-grupo5.mudancas_climaticas.brasil_1990_2010` AS a
INNER JOIN
  `projetofinal-grupo5.mudancas_climaticas.area-florestal` AS b
ON
  a.Pais = b.Pais
GROUP BY
  a.Pais,
  b.AreaFlorestal_1990_1000ha_,
  b.AreaFlorestal_2010_1000ha_,
  b.Variacao_1990_2010,
  b.VarPercentual_1990_2010
'''


In [ ]:
#Criação de Tabela oriente_medio_1990_2010 no BigQuery

# esta tabela fornece um recorte específico de Região - ÁsiaOeste/Oriente Médio - e período de tempo - 1990 a 2010
# para que possamos fazer análises cruzadas com os dados sobre área de cobertura florestal

'''
CREATE OR REPLACE TABLE
  mudancas_climaticas.oriente_medio_1990_2010 AS (
  SELECT
    Pais,
    Regiao,
    ROUND(SUM(DifTempMediaAno),5) AS VarTotalTempMedia,
    ROUND(SUM(EmissaoCO2TonAno),5) AS SomaEmissaoCO2
  FROM
    `projetofinal-grupo5.mudancas_climaticas.temp_co2`
  WHERE
    Regiao = 'Ásia Oeste'
    AND Ano >= 1990
    AND Ano <= 2010
  GROUP BY
    Pais,
    Regiao
  ORDER BY
    SomaEmissaoCO2 DESC
  LIMIT
    50 )
'''

In [ ]:
#Query 13

# esta query, que combina dados da tabela específica sobre Ásia Oeste/Oriente Médio no período de 1990 a 2010, 
# com dados da tabela sobre cobertura florestal no mundo entre 1990 e 2010, mostra uma possível forte correlação
# entre a falta de cobertura florestal e altos níveis de emissão de CO2 com a variação sensível de temperatura
# nesta região do planeta

spark.sql('''
SELECT 
  a.*,
  b.AreaFlorestal_1990_1000ha_,
  b.AreaFlorestal_2010_1000ha_,
  b.Variacao_1990_2010,
  b.VarPercentual_1990_2010
FROM 
  `projetofinal-grupo5.mudancas_climaticas.oriente_medio_1990_2010` AS a
INNER JOIN
  `projetofinal-grupo5.mudancas_climaticas.area-florestal` AS b
ON
  a.Pais = b.Pais
''').show()

In [ ]:
#Query 14 (BR)

# esta query mostra os valores mínimos e máximos de emissão de CO2 ton/ano 
# atingidas pelo Brasil dentro do recorte histórico de 1900 a 2012,
#que coincidem com os anos de 1900 e 2012, demonstrando uma tendência de crescente neste sentido
#desde o começo do perído analisado

spark.sql('''
SELECT 
  Pais, 
  Continente, 
  Regiao, 
  MIN(EmissaoCO2TonAno) AS minEmissaoCO2, 
  MAX(EmissaoCO2TonAno) AS maxEmissaoCO2
FROM 
  temp_co2
GROUP BY 
  Pais, 
  Continente, 
  Regiao
HAVING 
  Pais = 'Brazil'
''').show()

+------+----------+--------------+-------------+-------------+
|  Pais|Continente|        Regiao|minEmissaoCO2|maxEmissaoCO2|
+------+----------+--------------+-------------+-------------+
|Brazil|   América|América do Sul|    2103136.0| 4.60063104E8|
+------+----------+--------------+-------------+-------------+



In [ ]:
#Query 15

# esta query mostra o total das variações de temperatura média na região da América do Norte
# dentro de todo o período analisado - 1900 a 2012
# desta tabela resultante podemos aferir que, quanto mais ao norte se localiza o país, maiores foram as variação de temperatura 
# no período considerado

spark.sql('''
SELECT 
  Pais, 
  Regiao, 
  SUM(DifTempMediaAno) AS totalDifTempMedia
FROM 
  temp_co2
GROUP BY 
  Pais, 
  Regiao
HAVING
  Regiao = 'América do Norte'
ORDER BY 
  totalDifTempMedia DESC
''').show()

+-------------+----------------+------------------+
|         Pais|          Regiao| totalDifTempMedia|
+-------------+----------------+------------------+
|    Greenland|América do Norte|2.3875830620527267|
|       Canada|América do Norte|1.8419166505336764|
|United States|América do Norte|1.2394997378190364|
+-------------+----------------+------------------+

